In [ ]:
import numpy as np
from collections import Counter
import pandas as pd
from sklearn.metrics import accuracy_score

In [ ]:
col_names=['v_high','v_high.1','2','2.1','small','low','unacc']
data = pd.read_csv("car_evaluation.csv",skiprows=1,header=None,names=col_names)
data.head(10)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_v_high = LabelEncoder()
le_v_high_1 = LabelEncoder()
le_small = LabelEncoder()
le_low = LabelEncoder()
le_unacc = LabelEncoder()
le_2 = LabelEncoder()
le_21 = LabelEncoder()

In [ ]:
inputs = data.drop('unacc',axis='columns')
target = data['unacc']

In [ ]:
inputs['le_v_high'] = le_v_high.fit_transform(inputs['v_high'])
inputs['le_v_high_1'] = le_v_high_1.fit_transform(inputs['v_high.1'])
inputs['le_small'] = le_small.fit_transform(inputs['small'])
inputs['le_low'] = le_low.fit_transform(inputs['low'])
inputs['le_2'] = le_2.fit_transform(inputs['2'])
inputs['le_21'] = le_21.fit_transform(inputs['2.1'])
target = le_unacc.fit_transform(target)

In [ ]:
inputs

In [ ]:
inputs_n = inputs.drop(['v_high','v_high.1','small','low','2','2.1'],axis='columns')

In [ ]:
inputs_n = inputs_n.to_numpy()
# target = target.to_numpy()

In [ ]:
inputs_n

In [ ]:
def euclidean_distance(x1, x2):
    distance = np.sqrt(np.sum((x1-x2)**2))
    return distance

In [ ]:
class KNN:
    def __init__(self,k=3):
        self.k = k
    def fit(self,X,y):
        self.X_train = X
        self.y_train = y
    def predict(self,X):
        predictions = [self._predict(x) for x in X]
        return predictions
    def _predict(self,x):
        #compute the distance
        distances =[euclidean_distance(x,x_train) for x_train in self.X_train]

        #get the closest k
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels=[self.y_train[i] for i in k_indices]

        #majority vote
        most_common = Counter(k_nearest_labels).most_common()
        return most_common[0][0]
    

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(inputs_n, target, test_size=.1, random_state=0)

In [ ]:
X_train

In [ ]:
classifier=KNN(k=10)
classifier.fit(X_train,Y_train)
Y_pred = classifier.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)*100

In [ ]:
acc

In [ ]:
from sklearn.model_selection import KFold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=0)
accuracy_scores = []
classifiers=[]
for train_index, val_index in kf.split(inputs_n):
    X_train, X_val = inputs_n[train_index], inputs_n[val_index]
    Y_train, Y_val = target[train_index], target[val_index]
    classifier = KNN(k=5)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_val)
    acc = accuracy_score(Y_val, Y_pred) * 100
    accuracy_scores.append(acc)
    classifiers.append(classifier)

classifier=classifiers[np.argmax(accuracy_scores)]

In [ ]:
accuracy_scores

In [ ]:
y__pred=classifier.predict(X_test)
acc=accuracy_score(Y_test, y__pred)*100

In [ ]:
acc